In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2023-09-26 06:00:00')


In [3]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/97428
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2023-08-29 06:00:00 to 2023-09-26 05:00:00
2023-09-26 08:07:50,834 INFO: USE `taxi_demand_nyc_featurestore`
2023-09-26 08:07:51,300 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_demand_nyc_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2023-08-28 06:00:00.000' AND `fg0`.`pickup_hour` < TIMESTAMP '2023-09-27 05:00:00.000'


In [4]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/97428
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] bagging_fraction is set=0.9790939125249623, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9790939125249623


See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [5]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,1.0,2023-09-26 06:00:00
1,2,0.0,2023-09-26 06:00:00
2,3,0.0,2023-09-26 06:00:00
3,4,2.0,2023-09-26 06:00:00
4,5,0.0,2023-09-26 06:00:00
...,...,...,...
260,261,6.0,2023-09-26 06:00:00
261,262,71.0,2023-09-26 06:00:00
262,263,64.0,2023-09-26 06:00:00
263,264,23.0,2023-09-26 06:00:00


### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [6]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/97428
Connected. Call `.close()` to terminate connection gracefully.


In [7]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/97428/fs/97347/fg/115144


Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/97428/jobs/named/model_predictions_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x226ea7696d0>, None)